### Joel Brigida

### CDA4321: Cryptographic Engineering

### Lecture 5: Code for Assignment 2

### Multi-Precision Arithmetic:
 - If processor word size = 64-bit
     - Split a 256-bit number into a 4 index array of 64-bit numbers
 - If processor word size = 32-bit
     - Split a 256-but number into an 8 index array of 32-bit numbers

### $ res[0] = a[0] + b[0] $
### $ res[1] = a[1] + b[1] + \text{carry from } res[0] $
### $ res[2] = a[2] + b[2] + \text{carry from } res[1] $
### $ ... $

 - For a 256-bit number & 64-bit word size: use an array with 8 64-bit indices (Total allocation = 512 bits).
     - Every even index will contain 0's and used only to deal with the carry

In [1]:
result = 0x1234 - 0x4321
result2 = 0x1234 + (-0x4321)
print(f'{hex(0x1234)} - {hex(0x4321)} = {hex(result)}\n')
print(result.bits(), '\n')
print(hex(result2), '\n')
print(result.bits(), '\n')
bitList = list(result.bits())
print(bitList)

revBitList = bitList[::-1]
print(revBitList)

0x1234 - 0x4321 = -0x30ed

[-1, 0, -1, -1, 0, -1, -1, -1, 0, 0, 0, 0, -1, -1] 

-0x30ed 

[-1, 0, -1, -1, 0, -1, -1, -1, 0, 0, 0, 0, -1, -1] 

[-1, 0, -1, -1, 0, -1, -1, -1, 0, 0, 0, 0, -1, -1]
[-1, -1, 0, 0, 0, 0, -1, -1, -1, 0, -1, -1, 0, -1]


In [2]:
result3 = 0xFFFFCF13

print(result3.bits(), ' MSB on Right\n')
bitlist2 = result3.bits()
revBitList2 = bitlist2[::-1]
print(revBitList2, ' MSB on Left\n')

[1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]  MSB on Right

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1]  MSB on Left



In [3]:
a = 812937419827341987234918723501982375897410237 # A very large number
d = 4 # base 2^d: octal = 2^3 = 8, hex = 2^4 = 16
s = 4
print(a.digits(2^d), '\n') # print in octal or hex format

n = 4 # number of hex digits in each array index
hexArray = ["0x{0:0{1}x}".format(x,n) for x in a.digits(base=2^d,padto=s)]
print(hexArray, '\n')

v = 0xFFF7C3ED66
print(Integer(v))

[13, 11, 10, 2, 11, 0, 2, 10, 10, 4, 7, 6, 6, 14, 5, 3, 9, 7, 10, 5, 5, 12, 14, 5, 11, 6, 6, 13, 7, 8, 12, 5, 0, 1, 4, 7, 4, 2] 

['0x000d', '0x000b', '0x000a', '0x0002', '0x000b', '0x0000', '0x0002', '0x000a', '0x000a', '0x0004', '0x0007', '0x0006', '0x0006', '0x000e', '0x0005', '0x0003', '0x0009', '0x0007', '0x000a', '0x0005', '0x0005', '0x000c', '0x000e', '0x0005', '0x000b', '0x0006', '0x0006', '0x000d', '0x0007', '0x0008', '0x000c', '0x0005', '0x0000', '0x0001', '0x0004', '0x0007', '0x0004', '0x0002'] 

1099373473126


Source: https://nbviewer.org/urls/pastebin.com/raw/GJMCtHfA

Modifications Mine.

In [4]:
DATA_TYPE=32
def addn(a, b, r=DATA_TYPE): return (a + b) % (2^r)
def subn(a, b, r=DATA_TYPE): return (a - b) % (2^r)
def muln(a, b, r=DATA_TYPE): return (a * b) % (2^r)
def shift_leftn(a, b, r=DATA_TYPE):return (a >> b) % (2^r)
def shift_rightn(a, b, r=DATA_TYPE):return (a << b) % (2^r)
def andn(a, b, r=DATA_TYPE): return (a & b) % (2^r)
def orn(a, b, r=DATA_TYPE): return (a & b) % (2^r)
def xorn(a, b, r=DATA_TYPE): return (a.xor(b)) % (2^r)
def notn(a, r=DATA_TYPE): return (~a) % (2^r)

print(hex(subn(0x0, 0x2)), '\n')
print(hex(subn(0x1234, 0x4321)), '\n')

def print_integer_hex(a, base, padto, digits, s = ""):
    print(s + "[" + ','.join(["0x{0:0{1}x}".format(x,digits) for x in a.digits(base=base,padto=padto)]) + "]")

a = 0x123574fa
print_integer_hex(a, 2^8, 5, 4, "a = ")

0xfffffffe 

0xffffcf13 

a = [0x00fa,0x0074,0x0035,0x0012,0x0000]


Multiplication Schoolbook Method:
```python
             a3, a2, a1, a0
             b3, b2, b1, b0
            ---------------
     r6, r5, r4, r3, r2, r1

i = 0
 r0 =  a0 * b0   j = 0
 r1 =  a0 * b1   j = 1
 r2 =  a0 * b2   j = 2
 r3 =  a0 * b3   j = 3

i = 1
 r1 += a1 * b0
 r2 += a1 * b1
 r3 += a1 * b2
 r4 += a1 * b3

i = 2
 r2 += a2 * b0
 r3 += a2 * b1
 r4 += a2 * b2
 r5 += a2 * b3

```

In [ ]:
hex_string = 'F' * 64
hex_string = '0x' + hex_string
print(Integer(hex_string), '\n')

print(f'64 str F\'s: {hex_string}')
sageInt = Integer(hex_string)       # Requires SageMath: Convert String to Integer
print(f'64 int F\'s: {hex(sageInt)}\n')

Spare Operands:

In [ ]:
a = [ 0xB9, 0xDE, 0x02, 0x5B ]
b = [ 0x03, 0xEC, 0xFD, 0x41 ]

a = [ 0x5B, 0x02, 0xDE, 0xB9 ]
b = [ 0x41, 0xFD, 0xEC, 0x03 ]
r = [0, 0, 0, 0, 0, 0, 0]

Multiplication Comba Method:

In [ ]:
a1 = 0x5b02deb9
b1 = 0x41fdec03
c1 = 0

print(f'{hex(a1)} * {hex(b1)} = {hex(a1 * b1)}\n')

c1 = a1 * b1
print(f'{a1} * {b1} = {c1}\n')
d1 = hex(c1)
print(d1, '\n')

# a32 = 0x5b02deb9, a0 = b9, a1 = de, a2 = 02, a3 = 5b
# b32 = 0x41fdec03, b0 = 03, b1 = ec, b2 = fd, b3 = 41

a = [ 0x5B, 0x02, 0xDE, 0xB9 ]
b = [ 0x41, 0xFD, 0xEC, 0x03 ]
r = [0, 0, 0, 0, 0, 0, 0]

r[0] =  a[0] * b[0]
r[0] = hex(r[0] & 0xFFFF)

r[1] = (a[0] * b[1]) + (a[1] * b[0])
r[1] = hex(r[1] & 0xFFFF)

r[2] = (a[0] * b[2]) + (a[1] * b[1]) + (a[2] * b[0])
r[2] = hex(r[2] & 0xFFFF)

r[3] = (a[0] * b[3]) + (a[1] * b[2]) + (a[2] * b[1])
r[3] = hex(r[3] & 0xFFFF)

r[4] = (a[1] * b[3]) + (a[2] * b[2]) + (a[3] * b[1])
r[4] = hex(r[4] & 0xFFFF)

r[5] = (a[2] * b[3]) + (a[3] * b[2])
r[5] = hex(r[5] & 0xFFFF)

r[6] =  a[3] * b[3]
r[6] = hex(r[6] & 0xFFFF)

print(r)

Adding Hex Numbers in Sage and converting to a list of strings.

In [16]:
a = 0xabe89842fec093480d989c98a
b = 0x55f44c217f6049a406cc4e4c5

c = a * b
c.digits(base=2^16, padto=16)

print(f'In Hex: a * b = {hex(c)}\n')

d = 16
s = 16
n = 4

list16bit = ["0x{0:0{1}x}".format(x, n) for x in c.digits(base=2^d, padto=s)]

print(f'16 bit list: {list16bit}\n')

decResult = Integer(list16bit, base=2^d)
(f'In Decimal: a * b = {decResult}')

In Hex: a * b = 0x39b8475eea2950ed1e3b5e1e19698620966c1c3deaa42eff32

16 bit list: ['0xff32', '0xa42e', '0x3dea', '0x6c1c', '0x2096', '0x6986', '0x1e19', '0x3b5e', '0xed1e', '0x2950', '0x5eea', '0xb847', '0x0039', '0x0000', '0x0000', '0x0000']



'In Decimal: a * b = 362313301750115229922853852292127694260558623925754978565938'